In [1]:
import zipfile
%matplotlib inline 
import numpy as np 
import scipy as sp 
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.pyplot as plt
import pandas as pd 
import sklearn
import seaborn as sns
import os
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

In [2]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [5]:
events = pd.read_csv("events.csv")
events.head()

FileNotFoundError: [Errno 2] No such file or directory: 'events.csv'

In [4]:
events.info()

NameError: name 'events' is not defined

In [98]:
#Show data columns

events.columns

Index(['id_odsp', 'id_event', 'sort_order', 'time', 'text', 'event_type',
       'event_type2', 'side', 'event_team', 'opponent', 'player', 'player2',
       'player_in', 'player_out', 'shot_place', 'shot_outcome', 'is_goal',
       'location', 'bodypart', 'assist_method', 'situation', 'fast_break'],
      dtype='object')

In [99]:
# Find out the values of the "is goal column"
events["is_goal"].unique()

array([0, 1])

In [100]:
#find out the count of "is goal golumn"
events["is_goal"].value_counts()

0    916563
1     24446
Name: is_goal, dtype: int64

In [101]:
# Reading the dictionary data to see what event type are 

dictionary_text = open('dictionary.txt')

In [ ]:
dictionary_text.read()

'event_type\n0\tAnnouncement\n1\tAttempt\n2\tCorner\n3\tFoul\n4\tYellow card\n5\tSecond yellow card\n6\tRed card\n7\tSubstitution\n8\tFree kick won\n9\tOffside\n10\tHand ball\n11\tPenalty conceded\n\n\nevent_type2\n12\tKey Pass\n13\tFailed through ball\n14\tSending off\n15\tOwn goal\n\n\nside\n1\tHome\n2\tAway\n\n\nshot_place\n1\tBit too high\n2\tBlocked\n3\tBottom left corner\n4\tBottom right corner\n5\tCentre of the goal\n6\tHigh and wide\n7\tHits the bar\n8\tMisses to the left\n9\tMisses to the right\n10\tToo high\n11\tTop centre of the goal\n12\tTop left corner\n13\tTop right corner\n\n\nshot_outcome\n1\tOn target\n2\tOff target\n3\tBlocked\n4\tHit the bar\n\n\nlocation\n1\tAttacking half\n2\tDefensive half\n3\tCentre of the box\n4\tLeft wing\n5\tRight wing\n6\tDifficult angle and long range\n7\tDifficult angle on the left\n8\tDifficult angle on the right\n9\tLeft side of the box\n10\tLeft side of the six yard box\n11\tRight side of the box\n12\tRight side of the six yard box\n13\t

In [102]:
encoding = pd.read_table('dictionary.txt',  names=('num','events'))
encoding

,num,events
0,event_type,NaN
1,0,Announcement
2,1,Attempt
3,2,Corner
4,3,Foul
...,...,...
70,situation,NaN
71,1,Open play
72,2,Set piece
73,3,Corner


In [103]:
## checking the events column
events["event_type"].unique()

array([ 1,  2,  3,  8, 10,  7,  9,  4,  6, 11,  5])

In [104]:
#find out the count of "event_type"
events["event_type"].value_counts()

8     237932
3     232925
1     229135
2      91204
7      51738
9      43476
4      39911
10     10730
11      2706
6       1152
5        100
Name: event_type, dtype: int64

In [105]:
# We need the is_goal  column for events and the event_type when is equal 1 since that is a shot attempt.
shot_attempts = events[events["event_type"]== 1]

# We check the filter
shot_attempts.head() 

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
11,UFot0hit/,UFot0hit12,12,14,Attempt missed. Shinji Kagawa (Borussia Dortmu...,1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,13.0,2.0,0,15.0,1.0,1,1.0,0
13,UFot0hit/,UFot0hit14,14,17,"Goal! Borussia Dortmund 1, Hamburg 0. Kevin G...",1,12.0,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,4.0,1.0,1,9.0,2.0,1,1.0,0
14,UFot0hit/,UFot0hit15,15,19,Attempt blocked. Mats Hummels (Borussia Dortmu...,1,NaN,1,Borussia Dortmund,Hamburg SV,...,NaN,NaN,2.0,3.0,0,15.0,1.0,0,1.0,0
17,UFot0hit/,UFot0hit18,18,20,Attempt blocked. Tomas Rincon (Hamburg) right ...,1,NaN,2,Hamburg SV,Borussia Dortmund,...,NaN,NaN,2.0,3.0,0,15.0,1.0,0,1.0,0


In [106]:
### We'll check is the number is the same in both databases
shot_attempts["event_type"].value_counts()

1    229135
Name: event_type, dtype: int64

In [107]:
shot_attempts["player"].unique()

array(['mladen petric', 'shinji kagawa', 'kevin grosskreutz', ...,
       'alhassan wakaso', 'roman zozulia', 'alessandro bastoni'],
      dtype=object)

In [108]:
# Since is goal is a number meaning 1 is goal, we can sum the is goal column to get the top 50 scorers
filter_players= events.groupby(['player'])['is_goal'].sum().reset_index()
sorted_players = filter_players.sort_values('is_goal', ascending=False)
sorted_players.head(50)

,player,is_goal
3518,lionel messi,205
1215,cristiano ronaldo,198
6109,zlatan ibrahimovic,153
5074,robert lewandowski,124
1629,edinson cavani,121
2275,gonzalo higuain,118
4869,pierreemerick aubameyang,100
3660,luis suarez,96
1509,diego costa,93
312,alexandre lacazette,88


In [109]:
# Checking one player
neymar = (shot_attempts.player == 'neymar')
totalshots_neymar = shot_attempts.player[neymar].count()
totalshots_neymar


318

In [110]:
#check the ratio for the efectivness of Neymar
totalgoals_neymar = shot_attempts.player[neymar][shot_attempts.is_goal == 1].count()
effectivness_neymar = (totalgoals_neymar/totalshots_neymar) * 100

print('Neymar scored', totalgoals_neymar, 'goals with a total of', totalshots_neymar, 'for an efectivness of', effectivness_neymar.round(2), '%' )

Neymar scored 58 goals with a total of 318 for an efectivness of 18.24 %


In [111]:
# According to our dictionary database there are some shot locations that are not recorded (number 19)
print(shot_attempts.player[shot_attempts.location == 19].count()) 

1438


In [112]:
# Let's see the percentage. 
print(shot_attempts.event_type[shot_attempts.location == 19].count()/shot_attempts["event_type"].value_counts()* 100) 

1    0.632815
Name: event_type, dtype: float64


In [113]:
#Since is less than 1% we are going to drop those values, even though it may affect our results, it will have a minimal effect

shot_attempts = shot_attempts[shot_attempts.location != 19]


In [114]:
#cHECK NULL VALUIES
shot_attempts.isna().any()

id_odsp          False
id_event         False
sort_order       False
time             False
text             False
event_type       False
event_type2       True
side             False
event_team       False
opponent         False
player            True
player2           True
player_in         True
player_out        True
shot_place        True
shot_outcome     False
is_goal          False
location         False
bodypart         False
assist_method    False
situation        False
fast_break       False
dtype: bool

In [140]:
#Let's start our training and testing
# Create our features
X = shot_attempts[['time', 'side', 'bodypart', 'location', 'situation', 'assist_method', 'fast_break']]

# Create our target
y = shot_attempts['is_goal']



In [141]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(170763, 7)

In [142]:
X.describe()

,time,side,bodypart,location,situation,assist_method,fast_break
count,227685.000000,227685.000000,227685.000000,227685.000000,227685.000000,227685.000000,227685.000000
mean,49.219321,1.445045,1.627727,10.118089,1.273386,1.092176,0.020151
std,26.390378,0.496972,0.741302,5.194691,0.694405,0.933391,0.140516
min,0.000000,1.000000,1.000000,3.000000,1.000000,0.000000,0.000000
25%,27.000000,1.000000,1.000000,3.000000,1.000000,0.000000,0.000000
50%,50.000000,1.000000,1.000000,11.000000,1.000000,1.000000,0.000000
75%,72.000000,2.000000,2.000000,15.000000,1.000000,2.000000,0.000000
max,100.000000,2.000000,3.000000,18.000000,4.000000,4.000000,1.000000


In [143]:
# Check the balance of our target values
y.value_counts()

0    204682
1     23003
Name: is_goal, dtype: int64

In [121]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fit
brfc.fit(X_train, y_train)
print(X_train.shape)


(170763, 7)


In [122]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7032753782776697

In [147]:
# Check the balance of our target values
y.value_counts()

0    204682
1     23003
Name: is_goal, dtype: int64

In [149]:
# Display the confusion matrix
# YOUR CODE HERE
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["No_Goal", "is goal"], columns=["Predicted NO_Goal", "Predicted is goal"])
cm_df

,Predicted NO_Goal,Predicted is goal
No_Goal,35912,15259
is goal,1698,4053


In [123]:
# confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[35912, 15259],
       [ 1698,  4053]])

In [124]:

# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.95      0.70      0.70      0.81      0.70      0.49     51171
          1       0.21      0.70      0.70      0.32      0.70      0.49      5751

avg / total       0.88      0.70      0.70      0.76      0.70      0.49     56922



In [152]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 153511, 1: 153511})

In [171]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='lbfgs', random_state=1)
logreg

# Fit 
logreg.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [172]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
# Calculate predictions
y_pred = logreg.predict(X_test)

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.5989196298637909

In [174]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[31674, 19497],
       [ 2422,  3329]])

In [175]:
#create dataframe for confusion matrix
cm_df = pd.DataFrame(
    cm, index=["No_Goal", "is goal"], columns=["Predicted NO_Goal", "Predicted is goal"])
cm_df

,Predicted NO_Goal,Predicted is goal
No_Goal,31674,19497
is goal,2422,3329


In [176]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.62      0.58      0.74      0.60      0.36     51171
          1       0.15      0.58      0.62      0.23      0.60      0.36      5751

avg / total       0.85      0.61      0.58      0.69      0.60      0.36     56922

